# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

# Preprocessing the Data

In [2]:
# read csv into dataframe
filepath = "Austin Properties 2019.1.6.csv"
csv_df = pd.read_csv(filepath)

In [3]:
# check the column names
csv_df.columns

Index(['Address', 'Building Name', 'Zip Code', 'Bed', 'Bath', 'Avg SF',
       '# Units', 'Mix %', 'Units Available Units', 'Units Available Percent',
       'Avg Asking Rent/Unit', 'Avg Asking Rent/SF', 'Avg Effective Rent/Unit',
       'Avg Effective Rent/SF', 'Concessions %', 'Year Built',
       'Year Renovated', 'Units', 'Lande (Acres)', 'Vacancy %', 'Washer/Dryer',
       'Walk in Closet', 'Hardwood/Vinyl Floor', 'Walk Score', 'Transit Score',
       '1 Mile Population', '1 Mile Median Household Income',
       'Miles from Domain', 'Miles from Downtown', 'Unnamed: 29'],
      dtype='object')

In [4]:
csv_df = pd.get_dummies(data=csv_df, columns=['Washer/Dryer'])

In [5]:
# select certain variables
original_df = csv_df[['Bed', 'Bath', 'Avg SF', 'Concessions %', 'Year Built', 
   'Walk in Closet', 'Hardwood/Vinyl Floor',
   'Washer/Dryer_Yes but not in unit', 'Washer/Dryer_Yes in unit', 'Washer/Dryer_no',
   'Walk Score', 'Transit Score', '1 Mile Population', '1 Mile Median Household Income',
   'Miles from Domain', 'Miles from Downtown', 
   'Avg Effective Rent/Unit']]

In [6]:
# process na values
austin_properties_df = original_df.dropna()
# check the result of na value processing
austin_properties_df.count()

Bed                                 5468
Bath                                5468
Avg SF                              5468
Concessions %                       5468
Year Built                          5468
Walk in Closet                      5468
Hardwood/Vinyl Floor                5468
Washer/Dryer_Yes but not in unit    5468
Washer/Dryer_Yes in unit            5468
Washer/Dryer_no                     5468
Walk Score                          5468
Transit Score                       5468
1 Mile Population                   5468
1 Mile Median Household Income      5468
Miles from Domain                   5468
Miles from Downtown                 5468
Avg Effective Rent/Unit             5468
dtype: int64

In [7]:
# check the data type
austin_properties_df.dtypes

Bed                                 float64
Bath                                float64
Avg SF                              float64
Concessions %                       float64
Year Built                          float64
Walk in Closet                      float64
Hardwood/Vinyl Floor                float64
Washer/Dryer_Yes but not in unit      uint8
Washer/Dryer_Yes in unit              uint8
Washer/Dryer_no                       uint8
Walk Score                          float64
Transit Score                       float64
1 Mile Population                   float64
1 Mile Median Household Income      float64
Miles from Domain                   float64
Miles from Downtown                 float64
Avg Effective Rent/Unit             float64
dtype: object

In [8]:
# overview of the new dataframe
austin_properties_df.head()

,Bed,Bath,Avg SF,Concessions %,Year Built,Walk in Closet,Hardwood/Vinyl Floor,Washer/Dryer_Yes but not in unit,Washer/Dryer_Yes in unit,Washer/Dryer_no,Walk Score,Transit Score,1 Mile Population,1 Mile Median Household Income,Miles from Domain,Miles from Downtown,Avg Effective Rent/Unit
0,1.0,1.0,560.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1528.0
1,1.0,1.0,612.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1633.0
2,1.0,1.0,629.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1740.0
3,1.0,1.0,774.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1778.0
4,1.0,1.0,778.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1852.0


In [9]:
austin_properties_df["Year Built"].mean()

1997.693672275055

# Determine X and y for Machine Learning

In [10]:
X = austin_properties_df.iloc[:, :-1]
y = austin_properties_df['Avg Effective Rent/Unit']

# Splitting the data into training and testing sets

In [11]:
from sklearn.model_selection import train_test_split

# 80/20 training and testing data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [12]:
# check the shape of training and test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4374, 16)
(1094, 16)
(4374,)
(1094,)


# Model (I) - Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
# train the model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [15]:
# model evaluation
y_test_predicted = lin_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_predicted)
r2_test = r2_score(y_test, y_test_predicted)
print(f"Mean Squared Error (MSE): {mse_test}")
print(f"R-quared (R2): {r2_test}")

Mean Squared Error (MSE): 296512.9266368084
R-quared (R2): 0.5344241825987113


# Model (II) - Polynomial Regression

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [17]:
poly_model = Pipeline([
    ('a', PolynomialFeatures(degree=2)),
    ('b', LinearRegression(fit_intercept=False))
])

In [18]:
poly_model.fit(X_train, y_train)
poly_model.score(X_test, y_test)

0.44883007398009794

# Model (III) Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor()
param_grid = {'n_estimators' : list(range(1,999,2))}
model_rf = GridSearchCV(forest, param_grid, verbose=3)

model_rf.fit(X_train, y_train)
print(model_rf.best_params_)
print(model_rf.best_score_)

y_pred_rf = model_rf.predict(X_test)

/Users/apple/Documents/anaconda3/envs/PyData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


Fitting 3 folds for each of 499 candidates, totalling 1497 fits
[CV] n_estimators=1 ..................................................
[CV] .......... n_estimators=1, score=0.936103360114957, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8620350776106729, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8752445485214535, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.9343480771121119, total=   0.1s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.8845040595415848, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.8776823223594649, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ........

[CV] ........ n_estimators=39, score=0.9428164438377401, total=   0.7s
[CV] n_estimators=39 .................................................
[CV] ........ n_estimators=39, score=0.9314464511814243, total=   0.7s
[CV] n_estimators=39 .................................................
[CV] ........ n_estimators=39, score=0.9287848053424925, total=   0.6s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9430857404253309, total=   0.7s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9413825876316618, total=   0.7s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9227305347743747, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] ........ n_estimators=43, score=0.9449640438667319, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] .

[CV] ......... n_estimators=77, score=0.930502695942979, total=   1.2s
[CV] n_estimators=77 .................................................
[CV] ........ n_estimators=77, score=0.9355013357002917, total=   1.1s
[CV] n_estimators=79 .................................................
[CV] ........ n_estimators=79, score=0.9456394547686473, total=   1.1s
[CV] n_estimators=79 .................................................
[CV] ........ n_estimators=79, score=0.9322579683968085, total=   1.2s
[CV] n_estimators=79 .................................................
[CV] ........ n_estimators=79, score=0.9308673803336116, total=   1.3s
[CV] n_estimators=81 .................................................
[CV] ........ n_estimators=81, score=0.9406295110153147, total=   1.1s
[CV] n_estimators=81 .................................................
[CV] ........ n_estimators=81, score=0.9221225881965954, total=   1.1s
[CV] n_estimators=81 .................................................
[CV] .

[CV] ....... n_estimators=115, score=0.9323974845738037, total=   1.7s
[CV] n_estimators=117 ................................................
[CV] ....... n_estimators=117, score=0.9397100182573338, total=   1.7s
[CV] n_estimators=117 ................................................
[CV] ....... n_estimators=117, score=0.9315609773529102, total=   1.8s
[CV] n_estimators=117 ................................................
[CV] ....... n_estimators=117, score=0.9274499892984234, total=   1.7s
[CV] n_estimators=119 ................................................
[CV] ....... n_estimators=119, score=0.9464555603643081, total=   1.7s
[CV] n_estimators=119 ................................................
[CV] ....... n_estimators=119, score=0.9352693589315412, total=   1.7s
[CV] n_estimators=119 ................................................
[CV] ....... n_estimators=119, score=0.9291256370832028, total=   1.7s
[CV] n_estimators=121 ................................................
[CV] .

...

[CV] ....... n_estimators=889, score=0.9348265236010294, total=  12.9s
[CV] n_estimators=889 ................................................
[CV] ....... n_estimators=889, score=0.9302413975659783, total=  13.2s
[CV] n_estimators=891 ................................................
[CV] ....... n_estimators=891, score=0.9433153489603292, total=  12.4s
[CV] n_estimators=891 ................................................
[CV] ....... n_estimators=891, score=0.9354552123052465, total=  13.3s
[CV] n_estimators=891 ................................................
[CV] ....... n_estimators=891, score=0.9303359839102077, total=  13.6s
[CV] n_estimators=893 ................................................
[CV] ....... n_estimators=893, score=0.9423802362017597, total=  12.8s
[CV] n_estimators=893 ................................................
[CV] ....... n_estimators=893, score=0.9340042291471131, total=  12.8s
[CV] n_estimators=893 ................................................
[CV] .

[CV] ....... n_estimators=927, score=0.9310851403721302, total=  14.0s
[CV] n_estimators=929 ................................................
[CV] ....... n_estimators=929, score=0.9428782445588119, total=  13.2s
[CV] n_estimators=929 ................................................
[CV] ....... n_estimators=929, score=0.9361038823002242, total=  12.7s
[CV] n_estimators=929 ................................................
[CV] ....... n_estimators=929, score=0.9306554230248497, total=  14.2s
[CV] n_estimators=931 ................................................
[CV] ....... n_estimators=931, score=0.9424800300379463, total=  14.0s
[CV] n_estimators=931 ................................................
[CV] ....... n_estimators=931, score=0.9333670717319161, total=  14.1s
[CV] n_estimators=931 ................................................
[CV] ....... n_estimators=931, score=0.9306461960027953, total=  14.0s
[CV] n_estimators=933 ................................................
[CV] .

[CV] ....... n_estimators=967, score=0.9426098829534937, total=  13.4s
[CV] n_estimators=967 ................................................
[CV] ....... n_estimators=967, score=0.9356528611315482, total=  14.3s
[CV] n_estimators=967 ................................................
[CV] ....... n_estimators=967, score=0.9307960632032175, total=  15.6s
[CV] n_estimators=969 ................................................
[CV] ....... n_estimators=969, score=0.9430323657254022, total=  15.2s
[CV] n_estimators=969 ................................................
[CV] ....... n_estimators=969, score=0.9345112303549795, total=  14.3s
[CV] n_estimators=969 ................................................
[CV] ....... n_estimators=969, score=0.9307553210623376, total=  14.4s
[CV] n_estimators=971 ................................................
[CV] ....... n_estimators=971, score=0.9450099188476516, total=  14.6s
[CV] n_estimators=971 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done 1497 out of 1497 | elapsed: 185.7min finished


{'n_estimators': 189}
0.9387849851278849


In [21]:
import pickle
rf_model_filepath = '/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/Application/static/resources/RF999_Model.sav'
pickle.dump(model_rf, open(rf_model_filepath, 'wb'))

In [ ]:
model_rf.

In [ ]:
rf_model_filepath2 = '/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/Application/static/resources/Random_Forest_Model.sav'
loaded_model2 = pickle.load(open(rf_model_filepath2, 'rb'))

In [ ]:
loaded_model2.predict(X_new)

In [ ]:
loaded_model.predict(X_new)

In [23]:
X_new = [1, 1, 750, 0.02, 2018, 1, 1, 1, 0, 0, 100, 100, 70000, 50000, 0, 0]
X_new = pd.to_numeric(X_new)
X_new = np.reshape(X_new, (1, 16))
print(model_rf.predict(X_new))

[2750.38095238]


# Model (IV) - Neural Network Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [ ]:
print(len(X_train.columns))
print(X_train.columns)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# define neural network model
model_nn = Sequential()
model_nn.add(Dense(12, input_dim=16, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(8, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(4, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(1, kernel_initializer='random_uniform', activation='relu'))

# compile model
model_nn.compile(loss='mse', optimizer='adam')

In [ ]:
model_nn.fit(X_train, y_train, batch_size=300, epochs=10000, 
             verbose=1, 
             callbacks=[EarlyStopping(monitor='loss', min_delta=1000, patience=300)])

In [ ]:
filepath = "//weights.{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', 
                             verbose=0, save_best_only=True, 
                             save_weights_only=False, mode='min', period=100)

In [ ]:
y_predict = model_nn.predict(X_test)
print(r2_score(y_test, y_predict))

In [ ]:
X_new = [1, 1, 750, 0.02, 2018, 1, 1, 1, 0, 0, 100, 100, 70000, 50000, 0, 0]
X_new = pd.to_numeric(X_new)
X_new = np.reshape(X_new, (1, 16))

In [ ]:
model_nn.predict(X_new)

In [ ]:
X_train.columns

# Model (V) - K Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
k_neighbour = KNeighborsRegressor()

from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : list(range(1,999,2))}
model_k_neighbour = GridSearchCV(k_neighbour, param_grid, verbose=3)

model_k_neighbour.fit(X_train, y_train)

model_k_neighbour.best_params_

model_k_neighbour.best_score_

y_pred_k_neighbour = model_k_neighbour.predict(X_test)

# Residuals

In [ ]:
plt.scatter(y_test_predicted, y_test_predicted-y_test)
plt.hlines(y=0, xmin=y_test_predicted.min(), xmax=y_test_predicted.max())
plt.show()